In [17]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from tensorflow.keras import layers ,models
import numpy as np
import os

def load_X(path):
    X_signals = []
    files = os.listdir(path)
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.array(cell, dtype=np.float32) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    return np.transpose(np.array(X_signals), (1, 2, 0))#(totalStepNum*128*6)
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    y_ = y_ - 1
    #one_hot
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

X_train = load_X('gdrive/My Drive/Dataset_1/train/Inertial')
X_test = load_X('gdrive/My Drive/Dataset_1/test/Inertial')

train_label = load_y('gdrive/My Drive/Dataset_1/train/y_train.txt')
test_label = load_y('gdrive/My Drive/Dataset_1/test/y_test.txt')


Mounted at /content/drive
Mounted at /content/gdrive


In [18]:
print(X_train.shape)
print(X_test.shape)
print(train_label.shape)
print(test_label.shape)

dtrain_shape = X_train.shape
X_train = X_train.reshape(dtrain_shape[0], 1, dtrain_shape[1], dtrain_shape[2])
X_train = X_train.transpose(0, 3, 2, 1) 

dtest_shape = X_test.shape
X_test = X_test.reshape(dtest_shape[0], 1, dtest_shape[1], dtest_shape[2])
X_test = X_test.transpose(0, 3, 2, 1) 

print(X_train.shape)
print(X_test.shape)
print(train_label.shape)
print(test_label.shape)

num_classes = len(np.unique(train_label))
num_classes

(33104, 128, 6)
(3740, 128, 6)
(33104, 118)
(3740, 118)
(33104, 6, 128, 1)
(3740, 6, 128, 1)
(33104, 118)
(3740, 118)


2

In [19]:
idx = np.random.permutation(len(X_train))
X_train = X_train[idx]
train_label = train_label[idx]

In [20]:
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from tensorflow.keras import layers ,models
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM,ConvLSTM2D, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from keras.layers import concatenate
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [21]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], train_label.shape[1]
print( n_features,n_timesteps )

128 6


In [22]:
from keras.layers import Concatenate
model1_in = keras.layers.Input(X_train.shape[1:])
conv1_1 = keras.layers.Conv2D(filters=32, kernel_size=[1, 9], strides=[1,2], padding="same")(model1_in)
conv1_1 = keras.layers.ReLU()(conv1_1)
conv1_2 = Dropout(0.5)(conv1_1)
pool1 = keras.layers.MaxPooling2D(pool_size=(1, 2), strides=[1,2], padding="same", data_format=None)(conv1_2)  
conv2_1 = keras.layers.Conv2D(filters=64, kernel_size=[1, 3], strides=1,padding="same")(pool1)
conv2_1 = keras.layers.ReLU()(conv2_1)
conv2_2 = keras.layers.Conv2D(filters=128, kernel_size=[1, 3], strides=1, padding="same")(conv2_1)
conv2_2 = keras.layers.ReLU()(conv2_2)
pool2 = keras.layers.MaxPooling2D(pool_size=(1, 2), strides=[1,2], padding="same", data_format=None)(conv2_2)
conv3_1 = keras.layers.Conv2D(filters=128, kernel_size=[6, 1], strides=[6,1], padding="same")(pool2)
conv3_1 = keras.layers.ReLU()(conv3_1)
conv3_2 = Dropout(0.5)(conv3_1)
#model1_out= tf.reshape(conv3_1,[-1,n_outputs])
model1_outf = Flatten()(conv3_2)
cnn_model = Model(model1_in, model1_outf)
cnn_model.summary()

model2_in = keras.layers.Input(X_train.shape[1:])
l1 = keras.layers.TimeDistributed(LSTM(128,  activation='relu'))(model2_in)
l2 = keras.layers.TimeDistributed(Dropout(0.5))(l1)
l3 = keras.layers.TimeDistributed(Flatten())(l2)
l4 = keras.layers.LSTM(1024, activation = 'relu')(l3)  
l5 = Dropout(0.5)(l4)
model2_outf = Flatten()(l5)  
lstm_model = Model(model2_in, model2_outf)
lstm_model.summary()

inputs = tf.keras.layers.Input(shape=X_train.shape[1:])
combined = tf.keras.layers.Concatenate()([cnn_model(inputs), lstm_model(inputs)])
outputs = tf.keras.layers.Dense(118)(combined)

model3 = tf.keras.models.Model(inputs, outputs)
cnn_model.trainable = False
lstm_model.trainable = False




Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 6, 128, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 6, 64, 32)         320       
                                                                 
 re_lu_4 (ReLU)              (None, 6, 64, 32)         0         
                                                                 
 dropout_4 (Dropout)         (None, 6, 64, 32)         0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 32, 32)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 32, 64)         6208      
                                                           

In [28]:
checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
model3.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model3.fit(X_train, train_label, batch_size=1500, epochs=200, shuffle=True, 
             verbose=1, validation_split=0.2,
callbacks=[early_stopping, checkpoint])

Epoch 1/200
18/18 [==============================] - 14s 631ms/step - loss: 0.0222 - accuracy: 0.4368 - val_loss: 0.0081 - val_accuracy: 0.6632
Epoch 2/200
18/18 [==============================] - 10s 567ms/step - loss: 0.0090 - accuracy: 0.5677 - val_loss: 0.0069 - val_accuracy: 0.6809
Epoch 3/200
18/18 [==============================] - 10s 566ms/step - loss: 0.0071 - accuracy: 0.6486 - val_loss: 0.0065 - val_accuracy: 0.7861
Epoch 4/200
18/18 [==============================] - 10s 565ms/step - loss: 0.0067 - accuracy: 0.6753 - val_loss: 0.0064 - val_accuracy: 0.7950
Epoch 5/200
18/18 [==============================] - 10s 564ms/step - loss: 0.0067 - accuracy: 0.6821 - val_loss: 0.0064 - val_accuracy: 0.8040
Epoch 6/200
18/18 [==============================] - 10s 564ms/step - loss: 0.0067 - accuracy: 0.6834 - val_loss: 0.0064 - val_accuracy: 0.8068
Epoch 7/200
18/18 [==============================] - 10s 565ms/step - loss: 0.0067 - accuracy: 0.6841 - val_loss: 0.0064 - val_accuracy:

In [29]:
loss, accuracy = model3.evaluate(X_test, test_label, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Testing Accuracy: 0.7182
